In [1]:
import os
print(os.listdir())

['EmotionClassfication.ipynb', 'EmotionPrediction.ipynb', 'emotion_classifier_model.h5', 'emotion_classifier_model.keras', 'maxlen.txt', 'tokenizer.pkl', 'TwitterComments.csv']


In [3]:
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pandas as pd

# === Step 1: Load the saved tokenizer ===
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# === Step 2: Load the saved maxlen ===
with open("maxlen.txt", "r") as f:
    maxlen = int(f.read())

# === Step 3: Load the trained model ===
model = load_model("emotion_classifier_model.h5")

# === Step 4: Input text to classify ===
input_text = "I can't believe you did that! I can't even look at you"

# === Step 5: Preprocess input text ===
sequence = tokenizer.texts_to_sequences([input_text])
padded_sequence = pad_sequences(sequence, maxlen=maxlen, padding='post')

# === Step 6: Predict emotion ===
pred_probs = model.predict(padded_sequence)
predicted_label_index = np.argmax(pred_probs, axis=1)[0]

# === Step 7: Optional — Map index to emotion label ===
label_to_emotion = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

predicted_emotion = label_to_emotion.get(predicted_label_index, "unknown")

print("Predicted Emotion:", predicted_emotion)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 516ms/step
Predicted Emotion: sadness


In [ ]:
# === Read the CSV ===
df = pd.read_csv(r"D:\Character-and-Sentiment-Aware-Text-to-Speech-for-Expressive-Storytelling\EmotionClassification\dialogue_output.csv")  # Update with your actual file name

# === Define emotion classifier ===
def classify_emotion(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=maxlen, padding='post')
    pred_probs = model.predict(padded, verbose=0)
    label = np.argmax(pred_probs, axis=1)[0]
    return label_to_emotion.get(label, "unknown")

# === Apply emotion classification ===
def assign_emotion(row):
    if str(row["speaker"]).strip().lower() == "narrator":
        return "Neutral"
    else:
        return classify_emotion(row["text"])

# Add new column
df["emotions"] = df.apply(assign_emotion, axis=1)
# Save the updated dataframe
df.to_csv("quotes_with_emotions.csv", index=False)

print("✅ All dialogues classified — saved to 'quotes_with_emotions.csv'")



✅ All dialogues classified — saved to 'quotes_with_emotions.csv'


In [9]:
df.head()

,speaker,text,emotions
0,Narrator,"In a small town named Mathville, lived two bes...",Neutral
1,Sammy,Solve -1147 = -11*a - 1213 for ‘a’. What is th...,joy
2,Narrator,"Algy, being good at numbers, decided to take u...",Neutral
3,Algy,"First, we need to get ‘a’ alone. To do that, l...",sadness
4,Narrator,"So, they added 1213 to -1147, making it 66.\n\...",Neutral
